In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import VGG19
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

upscaled_size = (64, 64)
x_train_upscaled = tf.image.resize(x_train, upscaled_size)
x_test_upscaled = tf.image.resize(x_test, upscaled_size)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

170498071/170498071 [==============================] - 6s 0us/step


In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

In [ ]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

for layer in base_model.layers[:-5]:
    layer.trainable = False

model = models.Sequential()

model.add(base_model)

model.add(layers.Flatten())

model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(10, activation='softmax'))

80134624/80134624 [==============================] - 3s 0us/step


In [ ]:
model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 2, 2, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 batch_normalization (Batch  (None, 256)               1024      
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                        

In [ ]:
history = model.fit(datagen.flow(x_train_upscaled, y_train, batch_size=32),
                    steps_per_epoch=len(x_train_upscaled) / 32,
                    epochs=10,
                    validation_data=(x_test_upscaled, y_test))

Epoch 1/10
1562/1562 [==============================] - 117s 69ms/step - loss: 1.6741 - accuracy: 0.4713 - val_loss: 0.9316 - val_accuracy: 0.6953
Epoch 2/10
1562/1562 [==============================] - 105s 67ms/step - loss: 1.1539 - accuracy: 0.6234 - val_loss: 0.8939 - val_accuracy: 0.7144
Epoch 3/10
1562/1562 [==============================] - 107s 68ms/step - loss: 0.9896 - accuracy: 0.6735 - val_loss: 0.7812 - val_accuracy: 0.7423
Epoch 4/10
1562/1562 [==============================] - 108s 69ms/step - loss: 0.9077 - accuracy: 0.7011 - val_loss: 0.7792 - val_accuracy: 0.7490
Epoch 5/10
1562/1562 [==============================] - 106s 68ms/step - loss: 0.8459 - accuracy: 0.7223 - val_loss: 0.6853 - val_accuracy: 0.7679
Epoch 6/10
1562/1562 [==============================] - 105s 67ms/step - loss: 0.7829 - accuracy: 0.7404 - val_loss: 0.6407 - val_accuracy: 0.7808
Epoch 7/10
1562/1562 [==============================] - 105s 67ms/step - loss: 0.7519 - accuracy: 0.7533 - val_loss: 0

In [ ]:
test_loss, test_acc = model.evaluate(x_test_upscaled, y_test)
print(f'Test accuracy: {test_acc}')

313/313 [==============================] - 6s 20ms/step - loss: 0.5460 - accuracy: 0.8147
Test accuracy: 0.8147000074386597
